In [ ]:
"""TODO:
    - Refine narrative
    - Make plots
    - Determine presentation form.
        - Kyle very partial to some kind of web app"""

In [1]:
import pandas as pd
import plotly as px
import numpy


In [2]:
### Lists
## Years common across data
years = ["2020", "2021", "2022", "2023"]
years = sorted(list(years))


In [14]:
### Read in data
## Read in disengagements data
dis_dfs = []
for year in years:
    try:
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-Autonomous-Vehicle-Disengagement-Reports-CSV.csv", encoding='utf-8')
    except FileNotFoundError:
        print(f"File for year {year} not found. Skipping...")
        continue
    except UnicodeDecodeError:
        # If utf-8 decoding fails, try another common encoding
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-Autonomous-Vehicle-Disengagement-Reports-CSV.csv", encoding='latin-1')
    dis_dfs.append(annual_disengagements)

disengagement_df = pd.concat(dis_dfs, ignore_index=True)

# TODO: Cast DATE column as datetype for disengagements

## Read in mileage data
mile_dfs = []
for year in years:
    try:
        annual_mileage = pd.read_csv(f"data/Mileage/{year}-Autonomous-Mileage-Reports-CSV.csv", encoding='utf-8')
    except FileNotFoundError:
        print(f"File for year {year} not found. Skipping...")
        continue
    except UnicodeDecodeError:
        # If utf-8 decoding fails, try another common encoding
        annual_mileage = pd.read_csv(f"data/Mileage/{year}-Autonomous-Mileage-Reports-CSV.csv", encoding='latin-1')
    annual_mileage.rename_column() 
    mile_dfs.append(annual_mileage)
    # INFO: each annual report has 22 cols

mileage_df = pd.concat(mile_dfs, ignore_index=True)
mileage_df.set_index('VIN NUMBER', inplace=True)



File for year 2020 not found. Skipping...
File for year 2020 not found. Skipping...


In [15]:
# What are "Unnamed" colums? Probably useless "reserved"
unnamed_cols = ['Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21']
for check in unnamed_cols:
    non_null_count = mileage_df[check].notna().sum()
    print(f'Number of non values in {check}: {non_null_count}')

# Empty. Drop 'em
mileage_df = mileage_df.drop(columns=unnamed_cols)


Number of non values in Unnamed: 17: 0
Number of non values in Unnamed: 18: 0
Number of non values in Unnamed: 19: 0
Number of non values in Unnamed: 20: 0
Number of non values in Unnamed: 21: 0


In [20]:
## Modify Mileage dataframe to keep month columns instead of MM/YYYY pair columns, and add year column

# Create a new dataframe with the desired columns and retain the original data
new_mileage = pd.DataFrame(columns=['Manufacturer', 'Permit Number', 'Annual Total of Disengagements', 
                                'YEAR', 'DEC', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JULY', 'AUG',
                                'SEP', 'OCT', 'NOV', 'ANNUAL TOTAL'])

# Iterate over each year, merge the relevant month data into the new dataframe, and retain the original data
for year in years:
    temp_df = mileage_df.filter(regex=year)
    temp_df.columns = temp_df.columns.str.replace(year, "")
    temp_df["YEAR"] = year
    temp_df = pd.concat([mileage_df[mileage_df.columns.difference(temp_df.columns)], temp_df], axis=1)
    new_mileage = pd.concat([new_mileage, temp_df], axis=0, ignore_index=False)



C:\Users\outdo\AppData\Local\Temp\ipykernel_5532\3502773567.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df["YEAR"] = year
C:\Users\outdo\AppData\Local\Temp\ipykernel_5532\3502773567.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_mileage = pd.concat([new_mileage, temp_df], axis=0, ignore_index=False)
C:\Users\outdo\AppData\Local\Temp\ipykernel_5532\3502773567.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [21]:
cols_to_keep = ['Manufacturer', 'Permit Number',
        'Annual Total of Disengagements', 'YEAR', 'ANNUAL TOTAL',
        'DEC ', 'JAN ', 'FEB ', 'MAR ', 'APR ', 'MAY ', 'JUN ',
        'JULY ', 'AUG ', 'SEP ', 'OCT ', 'NOV ']

new_mileage = new_mileage[cols_to_keep]
print(new_mileage.columns)

Index(['Manufacturer', 'Permit Number', 'Annual Total of Disengagements',
       'YEAR', 'ANNUAL TOTAL', 'DEC ', 'JAN ', 'FEB ', 'MAR ', 'APR ', 'MAY ',
       'JUN ', 'JULY ', 'AUG ', 'SEP ', 'OCT ', 'NOV '],
      dtype='object')


In [22]:
pd.set_option('display.max_columns', None)

# Now when you use head(), it will display all columns
new_mileage.head()

# NOTE: Working on ANNUAL TOTALs , column didn't make it through the transformation properly
# I think everything else is correct.






,Manufacturer,Permit Number,Annual Total of Disengagements,YEAR,ANNUAL TOTAL,DEC,JAN,FEB,MAR,APR,MAY,JUN,JULY,AUG,SEP,OCT,NOV
JTDKN3DU9A0059509,AIMOTIVE INC.,AVT036,57.0,2020,1827,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4T1B21HK6KU514747,AIMOTIVE INC.,AVT036,49.0,2020,1149,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3LN6L5LU2HR615153,APOLLO AUTONOMOUS DRIVING USA LLC,AVT015,1.0,2020,830 .04,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3LN6L2LU4GR629715,APOLLO AUTONOMOUS DRIVING USA LLC,AVT015,0.0,2020,637.46,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3LN6L5LU1HR627987,APOLLO AUTONOMOUS DRIVING USA LLC,AVT015,0.0,2020,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
print(new_mileage.columns)

In [ ]:
#### Histogram: miles driven / VIN ######



In [ ]:
### Check if any manufacturers have multiple permit numbers?

In [ ]:
#### Bar: Total Miles driven per year ####
# Optional: groupby company ..."Manufacturer"
px.bar(
    mileage_df,
    x="Year",
    y="ANNUAL TOTAL",
    color = "Manufacturer"
)



In [ ]:
#### DISENGAGEMENTS PER MILE DRIVEN
# Makes most sense to show by year on a per-company basis. 
# Per vehicle basis doesn't make much sense (since we do not know anything about their operations)


In [ ]:
# INFO: mileage_df.shape = (4360, 46)
print(mileage_df.columns)
mileage_df.head()


In [ ]:
# INFO: disengagement_df.shape = (23579, 10)
print(disengagement_df.columns)